In [ ]:
!pip install tensorflow 

# Loading spacy's Norwegian and English languages 
!pip install -U spacy
!python -m spacy download en_core_web_sm

In [ ]:
!python -m spacy download nb_core_news_sm

In [3]:
import numpy as np
import pandas as pd
import nb_core_news_sm
import en_core_web_sm
import tensorflow as tf
tf.__version__

'2.12.0'

In [4]:
doc = pd.read_csv("nob.txt", delimiter="\t", header=None)
doc.head()

,0,1,2
0,Hi.,Hei,CC-BY 2.0 (France) Attribution: tatoeba.org #5...
1,Run!,Løp!,CC-BY 2.0 (France) Attribution: tatoeba.org #9...
2,Who?,Hvem?,CC-BY 2.0 (France) Attribution: tatoeba.org #2...
3,Wow!,Oi!,CC-BY 2.0 (France) Attribution: tatoeba.org #5...
4,Duck!,Dukk!,CC-BY 2.0 (France) Attribution: tatoeba.org #2...


In [7]:
del doc[doc. columns[-1]]

In [8]:
doc

,0,1
0,Hi.,Hei
1,Run!,Løp!
2,Who?,Hvem?
3,Wow!,Oi!
4,Duck!,Dukk!
...,...,...
7386,A great relationship is based on two main prin...,Et utmerket forhold er basert på to hovedprins...
7387,"There are many, many nuances in any language w...",Det er utallige nyanser i ethvert språk som er...
7388,Don't lend books; no one gives them back. The ...,Ikke lån vekk bøker; ingen gir dem tilbake. De...
7389,Words must be arranged in the proper sequence ...,Ord må ordnes i rett rekkefølge og grammatikkr...


In [9]:
doc.shape

(7391, 2)

In [10]:
doc=doc.sample(5000)

In [11]:
nl_corpus = " ".join(doc.iloc[:, 1].to_list())
en_corpus = " ".join(doc.iloc[:, 0].to_list())

In [12]:
nlp_nl = nb_core_news_sm.load()
nlp_nl.max_length = len(nl_corpus)

nlp_en = en_core_web_sm.load()
nlp_en.max_length = len(en_corpus)

nl_doc = nlp_nl(nl_corpus)
en_doc = nlp_en(en_corpus)

In [13]:
doc["nl_tokens"] = doc.iloc[:, 1].apply(nlp_nl.tokenizer)
doc["en_tokens"] = doc.iloc[:, 0].apply(nlp_en.tokenizer)

In [14]:
doc.tail()

,0,1,nl_tokens,en_tokens
3002,"It's difficult, isn't it?","Det er vanskelig, er det ikke?","(Det, er, vanskelig, ,, er, det, ikke, ?)","(It, 's, difficult, ,, is, n't, it, ?)"
530,Are you serious?,Er du seriøs?,"(Er, du, seriøs, ?)","(Are, you, serious, ?)"
2551,Don't try to deceive me.,Ikke prøv å lure meg.,"(Ikke, prøv, å, lure, meg, .)","(Do, n't, try, to, deceive, me, .)"
4883,We all have to be very careful.,Vi må alle være veldig forsiktige.,"(Vi, må, alle, være, veldig, forsiktige, .)","(We, all, have, to, be, very, careful, .)"
3359,She looked at the picture.,Hun så på bildet.,"(Hun, så, på, bildet, .)","(She, looked, at, the, picture, .)"


In [15]:
en_tokens = [token.text for token in en_doc]
en_vocabulary_set= set(en_tokens)
en_vocab_size = len(en_vocabulary_set)
print(en_vocab_size)

3333


In [16]:
nl_tokens = [token.text for token in nl_doc]
nl_vocabulary_set= set(nl_tokens)
nl_vocab_size = len(nl_vocabulary_set)
print(nl_vocab_size)

4099


In [17]:
all_en_tokens = {en_token: i + 1 for i, en_token in enumerate(en_vocabulary_set)}
all_nl_tokens = {nl_token: i + 1 for i, nl_token in enumerate(nl_vocabulary_set)}

In [18]:
def en_tokens_to_index(tokens):
    return [all_en_tokens[token.text] for token in tokens]

def nl_tokens_to_index(tokens):
    return [all_nl_tokens[token.text] for token in tokens]

In [19]:
doc["nl_indices"] = doc["nl_tokens"].apply(nl_tokens_to_index)
doc["en_indices"] = doc["en_tokens"].apply(en_tokens_to_index)

In [20]:
doc.tail()

,0,1,nl_tokens,en_tokens,nl_indices,en_indices
3002,"It's difficult, isn't it?","Det er vanskelig, er det ikke?","(Det, er, vanskelig, ,, er, det, ikke, ?)","(It, 's, difficult, ,, is, n't, it, ?)","[1330, 4077, 996, 596, 4077, 815, 2818, 3597]","[2716, 3096, 2619, 496, 3266, 1414, 2500, 2948]"
530,Are you serious?,Er du seriøs?,"(Er, du, seriøs, ?)","(Are, you, serious, ?)","[2891, 2536, 1214, 3597]","[3320, 3019, 2555, 2948]"
2551,Don't try to deceive me.,Ikke prøv å lure meg.,"(Ikke, prøv, å, lure, meg, .)","(Do, n't, try, to, deceive, me, .)","[3207, 2606, 572, 1789, 1592, 2061]","[2906, 1414, 270, 1013, 70, 522, 1676]"
4883,We all have to be very careful.,Vi må alle være veldig forsiktige.,"(Vi, må, alle, være, veldig, forsiktige, .)","(We, all, have, to, be, very, careful, .)","[2540, 113, 4, 2211, 1387, 912, 2061]","[2411, 3208, 610, 1013, 862, 1177, 226, 1676]"
3359,She looked at the picture.,Hun så på bildet.,"(Hun, så, på, bildet, .)","(She, looked, at, the, picture, .)","[1806, 1374, 2369, 125, 2061]","[2207, 1338, 636, 254, 2780, 1676]"


In [21]:
padded_nl_indices = tf.keras.preprocessing.sequence.pad_sequences(doc["nl_indices"], padding="post")
padded_en_indices = tf.keras.preprocessing.sequence.pad_sequences(doc["en_indices"], padding="post")

In [23]:
padded_nl_indices.shape

(5000, 27)

In [24]:
padded_en_indices.shape

(5000, 31)

In [25]:
doc["nl_indices"].apply(len).max()

27

In [26]:
doc["en_indices"].apply(len).max()

31

In [27]:
binarized_en_indices = tf.keras.utils.to_categorical(padded_en_indices, num_classes=en_vocab_size+1)
binarized_en_indices.shape

(5000, 31, 3334)

In [28]:
nl_ds = tf.data.Dataset.from_tensor_slices(padded_nl_indices)
en_ds = tf.data.Dataset.from_tensor_slices(binarized_en_indices)

In [29]:
tf_ds = tf.data.Dataset.zip((nl_ds, en_ds))

In [30]:
next(iter(tf_ds))

(<tf.Tensor: shape=(27,), dtype=int32, numpy=
 array([1330, 1595, 2548, 3166, 3628,   10, 2061,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0])>,
 <tf.Tensor: shape=(31, 3334), dtype=float32, numpy=
 array([[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [1., 0., 0., ..., 0., 0., 0.],
        [1., 0., 0., ..., 0., 0., 0.],
        [1., 0., 0., ..., 0., 0., 0.]], dtype=float32)>)

In [31]:
BATCH_SIZE = 32

tf_ds = tf_ds.shuffle(len(doc)).batch(BATCH_SIZE)

In [32]:
TAKE_SIZE = int(0.7 * len(doc) / BATCH_SIZE)

train_data = tf_ds.take(TAKE_SIZE)
test_data = tf_ds.skip(TAKE_SIZE)

In [34]:
model = tf.keras.Sequential([
                  # Input Word Embedding layer        
                  tf.keras.layers.Embedding(nl_vocab_size + 1, 64, mask_zero=True),

                  # LSTM Bidirectional layer
                  tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64, return_sequences=True)),
                  
                  # LSTM Bidirectionnal new layer
                  tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64, return_sequences=False)),

                  # Repeat Vector
                  tf.keras.layers.RepeatVector(binarized_en_indices.shape[1]),

                  # LSTM new layer
                  tf.keras.layers.LSTM(32, return_sequences=True),               

                  # Output layer with number of output neurons equal to class number with softmax function
                  tf.keras.layers.TimeDistributed(tf.keras.layers.Dense(en_vocab_size+1, activation="softmax"))
           
])

In [35]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, None, 64)          262400    
                                                                 
 bidirectional (Bidirectiona  (None, None, 128)        66048     
 l)                                                              
                                                                 
 bidirectional_1 (Bidirectio  (None, 128)              98816     
 nal)                                                            
                                                                 
 repeat_vector (RepeatVector  (None, 31, 128)          0         
 )                                                               
                                                                 
 lstm_2 (LSTM)               (None, 31, 32)            20608     
                                                        

In [36]:
input_text, output_text = next(iter(train_data))
print(input_text.numpy().shape)
print(model.predict(input_text).shape)
print(output_text.numpy().shape)

(32, 27)
1/1 [==============================] - 5s 5s/step
(32, 31, 3334)
(32, 31, 3334)


In [37]:
initial_learning_rate = 0.001

lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate,
    decay_steps=1090,
    decay_rate=0.96,
    staircase=True)

# Using a simple compiler with an Adam optimizer to compute our gradients 
optimizer= tf.keras.optimizers.Adam(
    learning_rate = lr_schedule
)

model.compile(optimizer=optimizer,
              loss=tf.keras.losses.CategoricalCrossentropy(),
              metrics=[tf.keras.metrics.CategoricalAccuracy()])

In [38]:
history = model.fit(train_data,
                    validation_data=test_data,
                    epochs=200)

Epoch 1/200
109/109 [==============================] - 39s 205ms/step - loss: 4.9957 - categorical_accuracy: 0.7592 - val_loss: 2.3753 - val_categorical_accuracy: 0.7667
Epoch 2/200
109/109 [==============================] - 17s 146ms/step - loss: 1.9917 - categorical_accuracy: 0.7659 - val_loss: 1.8473 - val_categorical_accuracy: 0.7672
Epoch 3/200
109/109 [==============================] - 17s 147ms/step - loss: 1.7441 - categorical_accuracy: 0.7670 - val_loss: 1.6184 - val_categorical_accuracy: 0.7690
Epoch 4/200
109/109 [==============================] - 16s 137ms/step - loss: 1.5829 - categorical_accuracy: 0.7662 - val_loss: 1.5347 - val_categorical_accuracy: 0.7663
Epoch 5/200
109/109 [==============================] - 16s 142ms/step - loss: 1.4949 - categorical_accuracy: 0.7681 - val_loss: 1.4717 - val_categorical_accuracy: 0.7664
Epoch 6/200
109/109 [==============================] - 17s 150ms/step - loss: 1.4560 - categorical_accuracy: 0.7673 - val_loss: 1.4488 - val_categoric

Epoch 49/200
109/109 [==============================] - 20s 177ms/step - loss: 1.0947 - categorical_accuracy: 0.8152 - val_loss: 1.1087 - val_categorical_accuracy: 0.8132
Epoch 50/200
109/109 [==============================] - 20s 176ms/step - loss: 1.0895 - categorical_accuracy: 0.8155 - val_loss: 1.0792 - val_categorical_accuracy: 0.8168
Epoch 51/200
109/109 [==============================] - 20s 174ms/step - loss: 1.0815 - categorical_accuracy: 0.8166 - val_loss: 1.0695 - val_categorical_accuracy: 0.8171
Epoch 52/200
109/109 [==============================] - 20s 178ms/step - loss: 1.0807 - categorical_accuracy: 0.8159 - val_loss: 1.0767 - val_categorical_accuracy: 0.8169
Epoch 53/200
109/109 [==============================] - 19s 171ms/step - loss: 1.0653 - categorical_accuracy: 0.8198 - val_loss: 1.0461 - val_categorical_accuracy: 0.8220
Epoch 54/200
109/109 [==============================] - 21s 190ms/step - loss: 1.0637 - categorical_accuracy: 0.8206 - val_loss: 1.0660 - val_cat

Epoch 97/200
109/109 [==============================] - 27s 234ms/step - loss: 0.9236 - categorical_accuracy: 0.8350 - val_loss: 0.9137 - val_categorical_accuracy: 0.8365
Epoch 98/200
109/109 [==============================] - 21s 179ms/step - loss: 0.9228 - categorical_accuracy: 0.8350 - val_loss: 0.8970 - val_categorical_accuracy: 0.8383
Epoch 99/200
109/109 [==============================] - 21s 182ms/step - loss: 0.9092 - categorical_accuracy: 0.8367 - val_loss: 0.8994 - val_categorical_accuracy: 0.8377
Epoch 100/200
109/109 [==============================] - 20s 178ms/step - loss: 0.9072 - categorical_accuracy: 0.8367 - val_loss: 0.8968 - val_categorical_accuracy: 0.8380
Epoch 101/200
109/109 [==============================] - 21s 185ms/step - loss: 0.9090 - categorical_accuracy: 0.8361 - val_loss: 0.8890 - val_categorical_accuracy: 0.8388
Epoch 102/200
109/109 [==============================] - 20s 180ms/step - loss: 0.9065 - categorical_accuracy: 0.8364 - val_loss: 0.9345 - val_

109/109 [==============================] - 20s 176ms/step - loss: 0.8141 - categorical_accuracy: 0.8467 - val_loss: 0.7968 - val_categorical_accuracy: 0.8487
Epoch 145/200
109/109 [==============================] - 19s 164ms/step - loss: 0.8001 - categorical_accuracy: 0.8495 - val_loss: 0.8056 - val_categorical_accuracy: 0.8474
Epoch 146/200
109/109 [==============================] - 19s 164ms/step - loss: 0.8120 - categorical_accuracy: 0.8467 - val_loss: 0.7967 - val_categorical_accuracy: 0.8495
Epoch 147/200
109/109 [==============================] - 19s 163ms/step - loss: 0.8006 - categorical_accuracy: 0.8488 - val_loss: 0.8016 - val_categorical_accuracy: 0.8480
Epoch 148/200
109/109 [==============================] - 19s 168ms/step - loss: 0.8022 - categorical_accuracy: 0.8484 - val_loss: 0.8005 - val_categorical_accuracy: 0.8482
Epoch 149/200
109/109 [==============================] - 19s 165ms/step - loss: 0.8054 - categorical_accuracy: 0.8477 - val_loss: 0.7792 - val_categorical

Epoch 192/200
109/109 [==============================] - 19s 170ms/step - loss: 0.7286 - categorical_accuracy: 0.8579 - val_loss: 0.7378 - val_categorical_accuracy: 0.8552
Epoch 193/200
109/109 [==============================] - 19s 167ms/step - loss: 0.7314 - categorical_accuracy: 0.8574 - val_loss: 0.7288 - val_categorical_accuracy: 0.8570
Epoch 194/200
109/109 [==============================] - 19s 169ms/step - loss: 0.7276 - categorical_accuracy: 0.8569 - val_loss: 0.7260 - val_categorical_accuracy: 0.8586
Epoch 195/200
109/109 [==============================] - 20s 171ms/step - loss: 0.7363 - categorical_accuracy: 0.8557 - val_loss: 0.7059 - val_categorical_accuracy: 0.8618
Epoch 196/200
109/109 [==============================] - 19s 169ms/step - loss: 0.7263 - categorical_accuracy: 0.8580 - val_loss: 0.6989 - val_categorical_accuracy: 0.8630
Epoch 197/200
109/109 [==============================] - 19s 166ms/step - loss: 0.7146 - categorical_accuracy: 0.8598 - val_loss: 0.7247 - v

In [39]:
for input_text, translation in test_data.take(1):
    pred = np.argmax(model.predict(input_text), axis=-1)

1/1 [==============================] - 4s 4s/step


In [41]:
indice_to_nl_token = dict(zip(all_nl_tokens.values(), all_nl_tokens.keys()))
indice_to_en_token = dict(zip(all_nl_tokens.values(), all_en_tokens.keys()))

In [42]:
for indice in input_text[0]:
    if indice == 0:
        break
    print(indice_to_nl_token[indice.numpy()])

Ditt
arbeid
er
langt
fra
tilfredsstillende
.


In [43]:
for indice in np.argmax(translation, axis=-1)[0]:
    if indice == 0:
        break
    print(indice_to_en_token[indice])

Your
work
is
far
from
being
satisfactory
.


In [44]:
for indice in pred[0]:
    if indice == 0:
        break
    print(indice_to_en_token[indice])

The
dog
is
is
is
is
is
.


In [45]:
history_2 = model.fit(train_data,
                      validation_data=test_data,
                      epochs=500)

Epoch 1/500
109/109 [==============================] - 18s 153ms/step - loss: 0.7183 - categorical_accuracy: 0.8587 - val_loss: 0.7338 - val_categorical_accuracy: 0.8546
Epoch 2/500
109/109 [==============================] - 18s 153ms/step - loss: 0.7055 - categorical_accuracy: 0.8613 - val_loss: 0.7094 - val_categorical_accuracy: 0.8601
Epoch 3/500
109/109 [==============================] - 18s 153ms/step - loss: 0.7206 - categorical_accuracy: 0.8577 - val_loss: 0.7153 - val_categorical_accuracy: 0.8594
Epoch 4/500
109/109 [==============================] - 18s 159ms/step - loss: 0.7159 - categorical_accuracy: 0.8592 - val_loss: 0.7106 - val_categorical_accuracy: 0.8595
Epoch 5/500
109/109 [==============================] - 21s 181ms/step - loss: 0.7139 - categorical_accuracy: 0.8596 - val_loss: 0.7192 - val_categorical_accuracy: 0.8574
Epoch 6/500
109/109 [==============================] - 20s 177ms/step - loss: 0.7138 - categorical_accuracy: 0.8586 - val_loss: 0.7145 - val_categoric

Epoch 49/500
109/109 [==============================] - 20s 174ms/step - loss: 0.6616 - categorical_accuracy: 0.8666 - val_loss: 0.6728 - val_categorical_accuracy: 0.8642
Epoch 50/500
109/109 [==============================] - 20s 171ms/step - loss: 0.6697 - categorical_accuracy: 0.8649 - val_loss: 0.6737 - val_categorical_accuracy: 0.8648
Epoch 51/500
109/109 [==============================] - 20s 176ms/step - loss: 0.6676 - categorical_accuracy: 0.8658 - val_loss: 0.6711 - val_categorical_accuracy: 0.8649
Epoch 52/500
109/109 [==============================] - 20s 174ms/step - loss: 0.6685 - categorical_accuracy: 0.8653 - val_loss: 0.6503 - val_categorical_accuracy: 0.8677
Epoch 53/500
109/109 [==============================] - 20s 174ms/step - loss: 0.6582 - categorical_accuracy: 0.8671 - val_loss: 0.6547 - val_categorical_accuracy: 0.8674
Epoch 54/500
109/109 [==============================] - 20s 172ms/step - loss: 0.6546 - categorical_accuracy: 0.8676 - val_loss: 0.6558 - val_cat

Epoch 97/500
109/109 [==============================] - 19s 165ms/step - loss: 0.6231 - categorical_accuracy: 0.8730 - val_loss: 0.6227 - val_categorical_accuracy: 0.8724
Epoch 98/500
109/109 [==============================] - 19s 167ms/step - loss: 0.6265 - categorical_accuracy: 0.8724 - val_loss: 0.6482 - val_categorical_accuracy: 0.8688
Epoch 99/500
109/109 [==============================] - 19s 170ms/step - loss: 0.6258 - categorical_accuracy: 0.8722 - val_loss: 0.6263 - val_categorical_accuracy: 0.8724
Epoch 100/500
109/109 [==============================] - 20s 171ms/step - loss: 0.6251 - categorical_accuracy: 0.8730 - val_loss: 0.6276 - val_categorical_accuracy: 0.8715
Epoch 101/500
109/109 [==============================] - 20s 172ms/step - loss: 0.6220 - categorical_accuracy: 0.8738 - val_loss: 0.6235 - val_categorical_accuracy: 0.8724
Epoch 102/500
109/109 [==============================] - 19s 167ms/step - loss: 0.6218 - categorical_accuracy: 0.8737 - val_loss: 0.6114 - val_

109/109 [==============================] - 19s 170ms/step - loss: 0.5864 - categorical_accuracy: 0.8802 - val_loss: 0.6005 - val_categorical_accuracy: 0.8775
Epoch 145/500
109/109 [==============================] - 19s 170ms/step - loss: 0.5878 - categorical_accuracy: 0.8793 - val_loss: 0.5797 - val_categorical_accuracy: 0.8813
Epoch 146/500
109/109 [==============================] - 20s 172ms/step - loss: 0.5794 - categorical_accuracy: 0.8813 - val_loss: 0.5673 - val_categorical_accuracy: 0.8823
Epoch 147/500
109/109 [==============================] - 20s 171ms/step - loss: 0.5839 - categorical_accuracy: 0.8803 - val_loss: 0.5824 - val_categorical_accuracy: 0.8814
Epoch 148/500
109/109 [==============================] - 19s 170ms/step - loss: 0.5875 - categorical_accuracy: 0.8796 - val_loss: 0.5802 - val_categorical_accuracy: 0.8824
Epoch 149/500
109/109 [==============================] - 19s 170ms/step - loss: 0.5839 - categorical_accuracy: 0.8800 - val_loss: 0.5815 - val_categorical

Epoch 192/500
109/109 [==============================] - 20s 176ms/step - loss: 0.5564 - categorical_accuracy: 0.8857 - val_loss: 0.5561 - val_categorical_accuracy: 0.8853
Epoch 193/500
109/109 [==============================] - 19s 169ms/step - loss: 0.5544 - categorical_accuracy: 0.8860 - val_loss: 0.5526 - val_categorical_accuracy: 0.8858
Epoch 194/500
109/109 [==============================] - 19s 170ms/step - loss: 0.5549 - categorical_accuracy: 0.8856 - val_loss: 0.5675 - val_categorical_accuracy: 0.8824
Epoch 195/500
109/109 [==============================] - 19s 170ms/step - loss: 0.5544 - categorical_accuracy: 0.8858 - val_loss: 0.5466 - val_categorical_accuracy: 0.8872
Epoch 196/500
109/109 [==============================] - 20s 171ms/step - loss: 0.5548 - categorical_accuracy: 0.8862 - val_loss: 0.5579 - val_categorical_accuracy: 0.8842
Epoch 197/500
109/109 [==============================] - 19s 169ms/step - loss: 0.5597 - categorical_accuracy: 0.8853 - val_loss: 0.5596 - v

...

Epoch 287/500
109/109 [==============================] - 20s 176ms/step - loss: 0.5104 - categorical_accuracy: 0.8939 - val_loss: 0.5182 - val_categorical_accuracy: 0.8931
Epoch 288/500
109/109 [==============================] - 21s 177ms/step - loss: 0.5150 - categorical_accuracy: 0.8930 - val_loss: 0.5255 - val_categorical_accuracy: 0.8912
Epoch 289/500
109/109 [==============================] - 19s 169ms/step - loss: 0.5094 - categorical_accuracy: 0.8940 - val_loss: 0.4934 - val_categorical_accuracy: 0.8980
Epoch 290/500
109/109 [==============================] - 19s 170ms/step - loss: 0.5185 - categorical_accuracy: 0.8922 - val_loss: 0.5141 - val_categorical_accuracy: 0.8932
Epoch 291/500
109/109 [==============================] - 20s 172ms/step - loss: 0.5172 - categorical_accuracy: 0.8930 - val_loss: 0.5281 - val_categorical_accuracy: 0.8925
Epoch 292/500
109/109 [==============================] - 19s 170ms/step - loss: 0.5167 - categorical_accuracy: 0.8932 - val_loss: 0.5028 - v

109/109 [==============================] - 21s 183ms/step - loss: 0.4982 - categorical_accuracy: 0.8963 - val_loss: 0.5114 - val_categorical_accuracy: 0.8926
Epoch 335/500
109/109 [==============================] - 20s 175ms/step - loss: 0.4947 - categorical_accuracy: 0.8977 - val_loss: 0.5022 - val_categorical_accuracy: 0.8951
Epoch 336/500
109/109 [==============================] - 20s 171ms/step - loss: 0.5015 - categorical_accuracy: 0.8956 - val_loss: 0.4894 - val_categorical_accuracy: 0.8989
Epoch 337/500
109/109 [==============================] - 19s 170ms/step - loss: 0.5000 - categorical_accuracy: 0.8959 - val_loss: 0.4933 - val_categorical_accuracy: 0.8973
Epoch 338/500
109/109 [==============================] - 20s 173ms/step - loss: 0.4959 - categorical_accuracy: 0.8969 - val_loss: 0.4853 - val_categorical_accuracy: 0.8987
Epoch 339/500
109/109 [==============================] - 20s 173ms/step - loss: 0.5010 - categorical_accuracy: 0.8960 - val_loss: 0.4845 - val_categorical

Epoch 382/500
109/109 [==============================] - 19s 170ms/step - loss: 0.4888 - categorical_accuracy: 0.8985 - val_loss: 0.4808 - val_categorical_accuracy: 0.9000
Epoch 383/500
109/109 [==============================] - 19s 169ms/step - loss: 0.4852 - categorical_accuracy: 0.8990 - val_loss: 0.4922 - val_categorical_accuracy: 0.8969
Epoch 384/500
109/109 [==============================] - 20s 173ms/step - loss: 0.4869 - categorical_accuracy: 0.8986 - val_loss: 0.4974 - val_categorical_accuracy: 0.8958
Epoch 385/500
109/109 [==============================] - 19s 169ms/step - loss: 0.4800 - categorical_accuracy: 0.8998 - val_loss: 0.4881 - val_categorical_accuracy: 0.8989
Epoch 386/500
109/109 [==============================] - 20s 173ms/step - loss: 0.4812 - categorical_accuracy: 0.8998 - val_loss: 0.4885 - val_categorical_accuracy: 0.8988
Epoch 387/500
109/109 [==============================] - 20s 174ms/step - loss: 0.4826 - categorical_accuracy: 0.9001 - val_loss: 0.4933 - v

109/109 [==============================] - 20s 173ms/step - loss: 0.4750 - categorical_accuracy: 0.9014 - val_loss: 0.4633 - val_categorical_accuracy: 0.9035
Epoch 430/500
109/109 [==============================] - 21s 181ms/step - loss: 0.4763 - categorical_accuracy: 0.9007 - val_loss: 0.4652 - val_categorical_accuracy: 0.9037
Epoch 431/500
109/109 [==============================] - 19s 169ms/step - loss: 0.4750 - categorical_accuracy: 0.9013 - val_loss: 0.4659 - val_categorical_accuracy: 0.9035
Epoch 432/500
109/109 [==============================] - 20s 173ms/step - loss: 0.4758 - categorical_accuracy: 0.9011 - val_loss: 0.4616 - val_categorical_accuracy: 0.9044
Epoch 433/500
109/109 [==============================] - 20s 173ms/step - loss: 0.4751 - categorical_accuracy: 0.9013 - val_loss: 0.4552 - val_categorical_accuracy: 0.9058
Epoch 434/500
109/109 [==============================] - 20s 173ms/step - loss: 0.4706 - categorical_accuracy: 0.9019 - val_loss: 0.4705 - val_categorical

Epoch 477/500
109/109 [==============================] - 19s 170ms/step - loss: 0.4657 - categorical_accuracy: 0.9028 - val_loss: 0.4756 - val_categorical_accuracy: 0.9019
Epoch 478/500
109/109 [==============================] - 20s 175ms/step - loss: 0.4618 - categorical_accuracy: 0.9044 - val_loss: 0.4671 - val_categorical_accuracy: 0.9032
Epoch 479/500
109/109 [==============================] - 19s 170ms/step - loss: 0.4676 - categorical_accuracy: 0.9030 - val_loss: 0.4545 - val_categorical_accuracy: 0.9052
Epoch 480/500
109/109 [==============================] - 19s 170ms/step - loss: 0.4717 - categorical_accuracy: 0.9016 - val_loss: 0.4609 - val_categorical_accuracy: 0.9042
Epoch 481/500
109/109 [==============================] - 20s 170ms/step - loss: 0.4624 - categorical_accuracy: 0.9032 - val_loss: 0.4603 - val_categorical_accuracy: 0.9044
Epoch 482/500
109/109 [==============================] - 20s 174ms/step - loss: 0.4654 - categorical_accuracy: 0.9034 - val_loss: 0.4621 - v

In [66]:
for input_text, translation in test_data.take(1):
    pred = np.argmax(model.predict(input_text), axis=-1)

# Norweigan 
print("Input Sentence:", end=" ")
for indice in input_text[2]:
    if indice == 0:
        break
    print(indice_to_nl_token[indice.numpy()], end=" ")

# True
print("\nTrue Translation:", end=" ")
for indice in np.argmax(translation, axis=-1)[2]:
    if indice == 0:
        break
    print(indice_to_en_token[indice], end=" ")

# Pred
print("\nModel Translation:", end=" ")
for indice in pred[2]:
    if indice == 0:
        break
    print(indice_to_en_token[indice], end=" ")

1/1 [==============================] - 0s 97ms/step
Input Sentence: Du er fremdeles involvert . 
True Translation: You 're still involved . 
Model Translation: You 're still involved . 

In [67]:
# Manually specify the input sentence
input_sentence = "Du er fremdeles involvert ."

input_tokens = input_sentence.split()
input_ids = [all_nl_tokens[token] for token in input_tokens]

max_length = 100 
input_ids_padded = input_ids[:max_length] + [0] * (max_length - len(input_ids))

input_text = np.array(input_ids_padded).reshape(1, -1)

pred = np.argmax(model.predict(input_text), axis=-1)

print("Input Sentence:", input_sentence)

print("\nNorwegian Translation:", end=" ")
for indice in pred[0]:
    if indice == 0:
        break
    print(indice_to_en_token[indice], end=" ")


1/1 [==============================] - 0s 51ms/step
Input Sentence: Du er fremdeles involvert .

Norwegian Translation: You 're still involved . 